# Layer Analytics
This notebook contains code to parse a saved TensorFlow model file and generate a pandas dataframe that contains information about all the layers.

In [1]:
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt
import plotly.express as px

# enable plotly in VS Studio Code
import plotly.io as pio
pio.renderers.default = "notebook_connected"

from workbench.utils.utils import parse_model_name

In [2]:
# Configure pandas to show all columns & rows
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [3]:
#model_name = "mobilenetv1_0.1_224_c3_o3_000"
model_name = "mobilenetv2_0.5_224_c3_o3_000"

In [4]:
parse_model_name(model_name)

'mobilenetv2'

In [5]:
filepath = f"i:/tinyml/tiny_cnn/models/{model_name}/{model_name}_layers.pkl"
df = pd.read_pickle(filepath)

In [6]:
df.columns

Index(['index', 'name', 'layer_type', 'input_shape', 'input', 'output_shape',
       'dtype', 'params', 'variables', 'weights_shape', 'weights_name',
       'weights_dtype', '#_activations', 'OPS', 'MACS', 'filters', 'trainable',
       'kernel_size', 'strides', 'padding', 'dilation_rate', 'groups',
       'activation', 'use_bias', 'data_format'],
      dtype='object')

In [60]:
df.head(10)

,index,name,layer_type,input_shape,input,output_shape,dtype,params,variables,weights_shape,weights_name,weights_dtype,#_activations,OPS,MACS,filters,trainable,kernel_size,strides,padding,dilation_rate,groups,activation,use_bias,data_format,input_1,input_2,b_i_1,h_i_1,w_i_1,c_i_1,b_i_2,h_i_2,w_i_2,c_i_2,#i_1_activations,#i_2_activations
0,0,input_1,<class 'keras.engine.input_layer.InputLayer'>,"(None, 224, 224, 3)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","[(None, 224, 224, 3)]",float32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(None, 224, 224, 3)",1,1.0,224,224,3,1.0,1,1,1,150528,0
1,1,conv2d,<class 'keras.layers.convolutional.conv2d.Conv...,"(None, 224, 224, 3)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 112, 112, 16)",float32,432,1,"(3, 3, 3, 16)",conv2d/kernel:0,<dtype: 'float32'>,200704,10838016,5419008,16,True,"(3, 3)","(2, 2)",same,"(1, 1)",1,linear,False,channels_last,"(None, 224, 224, 3)",1,1.0,224,224,3,1.0,1,1,1,150528,0
2,2,conv1_bn,<class 'keras.layers.normalization.batch_norma...,"(None, 112, 112, 16)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 112, 112, 16)",float32,64,4,(16),conv1_bn/gamma:0,<dtype: 'float32'>,0,0,0,0,0,0,0,0,0,0,0,0,0,"(None, 112, 112, 16)",1,1.0,112,112,16,1.0,1,1,1,200704,0
3,3,conv1_relu,<class 'keras.layers.activation.relu.ReLU'>,"(None, 112, 112, 16)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 112, 112, 16)",float32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(None, 112, 112, 16)",1,1.0,112,112,16,1.0,1,1,1,200704,0
4,4,block_1_dw_conv,<class 'keras.layers.convolutional.depthwise_c...,"(None, 112, 112, 16)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 112, 112, 16)",float32,144,1,"(3, 3, 16, 1)",block_1_dw_conv/depthwise_kernel:0,<dtype: 'float32'>,200704,3612672,1806336,0,True,"(3, 3)","(1, 1)",same,"(1, 1)",1,linear,False,channels_last,"(None, 112, 112, 16)",1,1.0,112,112,16,1.0,1,1,1,200704,0
5,5,block_1_dw_bn,<class 'keras.layers.normalization.batch_norma...,"(None, 112, 112, 16)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 112, 112, 16)",float32,64,4,(16),block_1_dw_bn/gamma:0,<dtype: 'float32'>,0,0,0,0,0,0,0,0,0,0,0,0,0,"(None, 112, 112, 16)",1,1.0,112,112,16,1.0,1,1,1,200704,0
6,6,block_1_dw_relu,<class 'keras.layers.activation.relu.ReLU'>,"(None, 112, 112, 16)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 112, 112, 16)",float32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(None, 112, 112, 16)",1,1.0,112,112,16,1.0,1,1,1,200704,0
7,7,block_1_compress,<class 'keras.layers.convolutional.conv2d.Conv...,"(None, 112, 112, 16)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 112, 112, 8)",float32,128,1,"(1, 1, 16, 8)",block_1_compress/kernel:0,<dtype: 'float32'>,100352,3211264,1605632,8,True,"(1, 1)","(1, 1)",same,"(1, 1)",1,linear,False,channels_last,"(None, 112, 112, 16)",1,1.0,112,112,16,1.0,1,1,1,200704,0
8,8,block_1_compress_bn,<class 'keras.layers.normalization.batch_norma...,"(None, 112, 112, 8)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 112, 112, 8)",float32,32,4,(8),block_1_compress_bn/gamma:0,<dtype: 'float32'>,0,0,0,0,0,0,0,0,0,0,0,0,0,"(None, 112, 112, 8)",1,1.0,112,112,8,1.0,1,1,1,100352,0
9,9,block_2_expand,<class 'keras.layers.convolutional.conv2d.Conv...,"(None, 112, 112, 8)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 112, 112, 48)",float32,384,1,"(1, 1, 8, 48)",block_2_expand/kernel:0,<dtype: 'float32'>,602112,9633792,4816896,48,True,"(1, 1)","(1, 1)",valid,"(1, 1)",1,linear,False,channels_last,"(None, 112, 112, 8)",1,1.0,112,112,8,1.0,1,1,1,100352,0


## Helper functions

In [25]:
def split_tuples(x):
    x = str(x).strip("()")
    x = x.split(",")
    return x

In [26]:
def return_batch_size(x):
    x = str(x).strip("()")
    split_strings = x.split(",")
    #print(split_strings)
    return split_strings[0].strip()

In [58]:
def return_input_height(x):
    #image_size=(img_height, img_width),

    x = str(x).strip("()")
    split_strings = x.split(",")
    if len(split_strings) in [3,4]:
        return split_strings[1].strip()
    else:
        return "None"


In [28]:
def return_input_width(x):
    #image_size=(img_height, img_width),

    x = str(x).strip("()")
    split_strings = x.split(",")
    if len(split_strings) in [4]:
        return split_strings[2].strip()
    else:
        return "None"


In [29]:
def return_channels(x):
    x = str(x).strip("()")
    split_strings = x.split(",")
    #print(split_strings)

    return split_strings[-1].strip()

In [11]:
def split_shape_columns(df, colum_name):
    df[colum_name] = df[colum_name].map(str)
    df[colum_name] = df[colum_name].str.strip("[]").replace('\)\,', ')*', regex=True)
    df_cols = df[colum_name].str.split("*", expand=True)
    prefix = colum_name.split("_")[0]
    df_cols.columns = [f"{prefix}_{x+1}" for x in df_cols.columns]
    return df_cols

# Input shapes & activations

In [23]:
df_input = split_shape_columns(df, "input_shape")
df_input

,input_1,input_2
0,"(None, 224, 224, 3)",None
1,"(None, 224, 224, 3)",None
2,"(None, 112, 112, 16)",None
3,"(None, 112, 112, 16)",None
4,"(None, 112, 112, 16)",None
...,...,...
150,"(None, 7, 7, 1280)",None
151,"(None, 1, 1, 1280)",None
152,"(None, 1, 1, 1280)",None
153,"(None, 1, 1, 3)",None


In [24]:
# #df["input_shape"][0] =(None, None, None, None)
# df["input_shape"] = df["input_shape"].map(str)
# df["input_shape"] = df["input_shape"].str.strip("[]").replace('\)\,', ')*', regex=True)
# df_input = df["input_shape"].str.split("*", expand=True)
# df_input.columns =[f"input_{x+1}" for x in df_input.columns]
# no_input_cols = len(df_input.columns)
# print(no_input_cols)
# df_input


In [30]:
#df_input["input_1_split"] = df_input["input_1"].apply(split_tuples)

In [31]:
no_input_cols = len(df_input.columns)

In [32]:
for input in range(1,no_input_cols+1):
    df_input[f"b_i_{input}"]= df_input[f"input_{input}"].apply(return_batch_size)
    df_input[f"h_i_{input}"] = df_input[f"input_{input}"].apply(return_input_height)
    df_input[f"w_i_{input}"] = df_input[f"input_{input}"].apply(return_input_width)
    df_input[f"c_i_{input}"] = df_input[f"input_{input}"].apply(return_channels)

df_input

,input_1,input_2,b_i_1,h_i_1,w_i_1,c_i_1,b_i_2,h_i_2,w_i_2,c_i_2
0,"(None, 224, 224, 3)",None,None,224,224,3,None,None,None,None
1,"(None, 224, 224, 3)",None,None,224,224,3,None,None,None,None
2,"(None, 112, 112, 16)",None,None,112,112,16,None,None,None,None
3,"(None, 112, 112, 16)",None,None,112,112,16,None,None,None,None
4,"(None, 112, 112, 16)",None,None,112,112,16,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
150,"(None, 7, 7, 1280)",None,None,7,7,1280,None,None,None,None
151,"(None, 1, 1, 1280)",None,None,1,1,1280,None,None,None,None
152,"(None, 1, 1, 1280)",None,None,1,1,1280,None,None,None,None
153,"(None, 1, 1, 3)",None,None,1,1,3,None,None,None,None


In [43]:
#df_input["w_i_2"].replace('None', np.nan, inplace=True)
df_input["b_i_1"].replace('None', np.nan, inplace=True)
df_input["w_i_1"].replace('None', np.nan, inplace=True)
df_input["h_i_1"].replace('None', np.nan, inplace=True)
df_input["c_i_1"].replace('None', np.nan, inplace=True)

df_input["b_i_2"] = df_input["b_i_2"].str.replace("\(", "")
df_input["b_i_2"].replace('None', np.nan, inplace=True)
df_input["w_i_2"].replace('None', np.nan, inplace=True)
df_input["h_i_2"].replace('None', np.nan, inplace=True)
df_input["c_i_2"].replace('None', np.nan, inplace=True)
df_input.fillna(1, inplace=True)


C:\Users\Susanne\AppData\Local\Temp\ipykernel_22392\2502296835.py:7: FutureWarning:

The default value of regex will change from True to False in a future version.



In [44]:
df_input

,input_1,input_2,b_i_1,h_i_1,w_i_1,c_i_1,b_i_2,h_i_2,w_i_2,c_i_2
0,"(None, 224, 224, 3)",1,1.0,224,224,3,1.0,1,1,1
1,"(None, 224, 224, 3)",1,1.0,224,224,3,1.0,1,1,1
2,"(None, 112, 112, 16)",1,1.0,112,112,16,1.0,1,1,1
3,"(None, 112, 112, 16)",1,1.0,112,112,16,1.0,1,1,1
4,"(None, 112, 112, 16)",1,1.0,112,112,16,1.0,1,1,1
...,...,...,...,...,...,...,...,...,...,...
150,"(None, 7, 7, 1280)",1,1.0,7,7,1280,1.0,1,1,1
151,"(None, 1, 1, 1280)",1,1.0,1,1,1280,1.0,1,1,1
152,"(None, 1, 1, 1280)",1,1.0,1,1,1280,1.0,1,1,1
153,"(None, 1, 1, 3)",1,1.0,1,1,3,1.0,1,1,1


In [61]:
#df_input.c_i_1.value_counts()

In [82]:

df_input["w_i_1"] = df_input["w_i_1"].map(int)
df_input["h_i_1"] = df_input["h_i_1"].map(int)
df_input["b_i_1"] = df_input["b_i_1"].map(int)
df_input["c_i_1"] = df_input["c_i_1"].map(int)

df_input["b_i_2"] = df_input["b_i_2"].map(int)
df_input["w_i_2"] = df_input["w_i_2"].map(int)
df_input["h_i_2"] = df_input["h_i_2"].map(int)
df_input["c_i_2"] = df_input["c_i_2"].map(int)

In [47]:

df_input["#i_1_activations"] = df_input["h_i_1"] * df_input["w_i_1"] * df_input["c_i_1"]


In [62]:
def calculate_i_2_activations(x):
    h_i = int(x["h_i_2"])#.map(int)
    w_i = int(x["w_i_2"])#.map(int)
    c_i = int(x["c_i_2"])#.map(int)
    out = h_i * w_i * c_i
    if out >1:
        return out
    else:
        return 0

In [63]:
df_input["#i_2_activations"] = df_input.apply(calculate_i_2_activations, axis=1)

In [64]:

# df_input["b_i_1"] = df_input["b_i_1"].map(int)
# df_input["w_i_1"] = df_input["w_i_1"].map(int)
# df_input["h_i_1"] = df_input["h_i_1"].map(int)
# df_input["c_i_1"] = df_input["c_i_1"].map(int)

# df_input["b_i_2"] = df_input["b_i_2"].map(int)
# df_input["w_i_2"] = df_input["w_i_2"].map(int)
# df_input["h_i_2"] = df_input["h_i_2"].map(int)
# df_input["c_i_2"] = df_input["c_i_2"].map(int)

In [65]:
df_input

,input_1,input_2,b_i_1,h_i_1,w_i_1,c_i_1,b_i_2,h_i_2,w_i_2,c_i_2,#i_1_activations,#i_2_activations
0,"(None, 224, 224, 3)",1,1.0,224,224,3,1.0,1,1,1,150528,0
1,"(None, 224, 224, 3)",1,1.0,224,224,3,1.0,1,1,1,150528,0
2,"(None, 112, 112, 16)",1,1.0,112,112,16,1.0,1,1,1,200704,0
3,"(None, 112, 112, 16)",1,1.0,112,112,16,1.0,1,1,1,200704,0
4,"(None, 112, 112, 16)",1,1.0,112,112,16,1.0,1,1,1,200704,0
...,...,...,...,...,...,...,...,...,...,...,...,...
150,"(None, 7, 7, 1280)",1,1.0,7,7,1280,1.0,1,1,1,62720,0
151,"(None, 1, 1, 1280)",1,1.0,1,1,1280,1.0,1,1,1,1280,0
152,"(None, 1, 1, 1280)",1,1.0,1,1,1280,1.0,1,1,1,1280,0
153,"(None, 1, 1, 3)",1,1.0,1,1,3,1.0,1,1,1,3,0


In [66]:
df_input["#i_2_activations"].value_counts()

0        145
6272       3
12544      2
9408       2
3920       2
37632      1
Name: #i_2_activations, dtype: int64

In [56]:
df = df.join(df_input)

# Output shape and activations

In [74]:
df_output = split_shape_columns(df, "output_shape")
df_output

,output_1
0,"(None, 224, 224, 3)"
1,"(None, 112, 112, 16)"
2,"(None, 112, 112, 16)"
3,"(None, 112, 112, 16)"
4,"(None, 112, 112, 16)"
...,...
150,"(None, 1, 1, 1280)"
151,"(None, 1, 1, 1280)"
152,"(None, 1, 1, 3)"
153,"(None, 1, 3)"


In [75]:
no_output_cols = len(df_output.columns)
no_output_cols

1

In [76]:
for output in range(1,no_output_cols+1):
    df_output[f"b_o_{output}"]= df_output[f"output_{output}"].apply(return_batch_size)
    df_output[f"h_o_{output}"] = df_output[f"output_{output}"].apply(return_input_height)
    df_output[f"w_o_{output}"] = df_output[f"output_{output}"].apply(return_input_width)
    df_output[f"c_o_{output}"] = df_output[f"output_{output}"].apply(return_channels)

df_output

,output_1,b_o_1,h_o_1,w_o_1,c_o_1
0,"(None, 224, 224, 3)",None,224,224,3
1,"(None, 112, 112, 16)",None,112,112,16
2,"(None, 112, 112, 16)",None,112,112,16
3,"(None, 112, 112, 16)",None,112,112,16
4,"(None, 112, 112, 16)",None,112,112,16
...,...,...,...,...,...
150,"(None, 1, 1, 1280)",None,1,1,1280
151,"(None, 1, 1, 1280)",None,1,1,1280
152,"(None, 1, 1, 3)",None,1,1,3
153,"(None, 1, 3)",None,1,None,3


In [79]:
df_output["b_o_1"].replace('None', np.nan, inplace=True)
df_output["w_o_1"].replace('None', np.nan, inplace=True)
df_output["h_o_1"].replace('None', np.nan, inplace=True)
df_output["c_o_1"].replace('None', np.nan, inplace=True)
df_output.fillna(1, inplace=True)

In [83]:
df_output["w_o_1"] = df_output["w_o_1"].map(int)
df_output["h_o_1"] = df_output["h_o_1"].map(int)
df_output["b_o_1"] = df_output["b_o_1"].map(int)
df_output["c_o_1"] = df_output["c_o_1"].map(int)

In [84]:
df_output["#o_1_activations"] = df_output["h_o_1"] * df_output["w_o_1"] * df_output["c_o_1"]

In [85]:
df = df.join(df_output)

In [88]:
df["peak_activations"] = df["#i_1_activations"] + df["#o_1_activations"]

In [89]:
df["total_params"] = df["params"].cumsum()

In [90]:
df.columns

Index(['index', 'name', 'layer_type', 'input_shape', 'input', 'output_shape',
       'dtype', 'params', 'variables', 'weights_shape', 'weights_name',
       'weights_dtype', '#_activations', 'OPS', 'MACS', 'filters', 'trainable',
       'kernel_size', 'strides', 'padding', 'dilation_rate', 'groups',
       'activation', 'use_bias', 'data_format', 'input_1', 'input_2', 'b_i_1',
       'h_i_1', 'w_i_1', 'c_i_1', 'b_i_2', 'h_i_2', 'w_i_2', 'c_i_2',
       '#i_1_activations', '#i_2_activations', 'output_1', 'b_o_1', 'h_o_1',
       'w_o_1', 'c_o_1', '#o_1_activations', 'total_params',
       'peak_activations'],
      dtype='object')

In [91]:
df.head()

,index,name,layer_type,input_shape,input,output_shape,dtype,params,variables,weights_shape,weights_name,weights_dtype,#_activations,OPS,MACS,filters,trainable,kernel_size,strides,padding,dilation_rate,groups,activation,use_bias,data_format,input_1,input_2,b_i_1,h_i_1,w_i_1,c_i_1,b_i_2,h_i_2,w_i_2,c_i_2,#i_1_activations,#i_2_activations,output_1,b_o_1,h_o_1,w_o_1,c_o_1,#o_1_activations,total_params,peak_activations
0,0,input_1,<class 'keras.engine.input_layer.InputLayer'>,"(None, 224, 224, 3)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 224, 224, 3)",float32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(None, 224, 224, 3)",1,1.0,224,224,3,1.0,1,1,1,150528,0,"(None, 224, 224, 3)",1,224,224,3,150528,0,301056
1,1,conv2d,<class 'keras.layers.convolutional.conv2d.Conv...,"(None, 224, 224, 3)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 112, 112, 16)",float32,432,1,"(3, 3, 3, 16)",conv2d/kernel:0,<dtype: 'float32'>,200704,10838016,5419008,16,True,"(3, 3)","(2, 2)",same,"(1, 1)",1,linear,False,channels_last,"(None, 224, 224, 3)",1,1.0,224,224,3,1.0,1,1,1,150528,0,"(None, 112, 112, 16)",1,112,112,16,200704,432,351232
2,2,conv1_bn,<class 'keras.layers.normalization.batch_norma...,"(None, 112, 112, 16)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 112, 112, 16)",float32,64,4,(16),conv1_bn/gamma:0,<dtype: 'float32'>,0,0,0,0,0,0,0,0,0,0,0,0,0,"(None, 112, 112, 16)",1,1.0,112,112,16,1.0,1,1,1,200704,0,"(None, 112, 112, 16)",1,112,112,16,200704,496,401408
3,3,conv1_relu,<class 'keras.layers.activation.relu.ReLU'>,"(None, 112, 112, 16)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 112, 112, 16)",float32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(None, 112, 112, 16)",1,1.0,112,112,16,1.0,1,1,1,200704,0,"(None, 112, 112, 16)",1,112,112,16,200704,496,401408
4,4,block_1_dw_conv,<class 'keras.layers.convolutional.depthwise_c...,"(None, 112, 112, 16)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 112, 112, 16)",float32,144,1,"(3, 3, 16, 1)",block_1_dw_conv/depthwise_kernel:0,<dtype: 'float32'>,200704,3612672,1806336,0,True,"(3, 3)","(1, 1)",same,"(1, 1)",1,linear,False,channels_last,"(None, 112, 112, 16)",1,1.0,112,112,16,1.0,1,1,1,200704,0,"(None, 112, 112, 16)",1,112,112,16,200704,640,401408


In [97]:
fig = px.bar(df, x="name", y= "peak_activations", text_auto=".2s",
    title=f"Peak activations per layer - {model_name}")
fig.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange", annotation_text="MCU constraint", 
              annotation_position="bottom right")
#fig.update_layout(showlegend=True)
fig.show()

In [95]:
fig2 = px.bar(df, x="name", y= "h_i_1", text_auto=".2s",
    title=f"Input height per layer - {model_name}")
#fig2.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange")
fig2.update_yaxes(autorange="reversed")
fig2.update_yaxes(rangemode="tozero")
fig2.show()

In [96]:
fig3 = px.bar(df, x="name", y= "c_i_1", text_auto=".2s",
    title=f"Channels per layer - {model_name}")
#fig2.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange")
#fig3.update_yaxes(autorange="reversed")
#fig3.update_yaxes(rangemode="tozero")
# fig3.update_yaxes(
#     range=(0, 1280),
#     constrain='domain'
# )
fig3.show()